# **Capstone Project**

## Leading Indicators
| File Name | Type | Use |
|-----------|------|-----|
| `Consumer_Confidence_Index.csv` | Leading | Feature |
| `Business_Confidence_Index.csv` | Leading | Feature |
| `Initial_Claims.csv` | Leading | Feature |
| `Jobs_Added.csv` | Leading | Feature |
| `Housing_Starts.csv` | Leading | Feature |
| `Yield_Curve.csv` | Leading | Feature |
| `Federal_Funds_Rate.csv` | Leading (policy-reactive) | Feature |
| `Supply_Chain_Index.xls` | Leading | Feature |
| `Crude_Oil_Prices.csv` | Contextual | Feature |

In [102]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
BCI_df = pd.read_csv('Leading/Business_Confidence_Index.csv')
CCI_df = pd.read_csv('Leading/Consumer_Confidence_Index.csv', engine='python', on_bad_lines='skip')
Oil_Prices_df = pd.read_csv('Leading/Crude_Oil_Prices.csv')
Interest_Rates_df = pd.read_csv('Leading/Interest_Rates.csv')
Yield_Curve_df = pd.read_csv('Leading/Yield_Curve.csv')
Housing_Starts_df = pd.read_csv('Leading/Housing_Starts.csv')
Jobless_Claims_df = pd.read_csv('Leading/Initial_Claims.csv')
Jobs_Added_df = pd.read_csv('Leading/Jobs_Added.csv')



# ---------------------------------------------------
# 🗃️ Load DataFrames into an SQLite in-memory database
# ---------------------------------------------------
conn = sqlite3.connect(':memory:')

BCI_df.to_sql('BusinessConfidence', conn, index=False, if_exists='replace')
CCI_df.to_sql('ConsumerConfidence', conn, index=False, if_exists='replace')
Oil_Prices_df.to_sql('OilPrices', conn, index=False, if_exists='replace')
Interest_Rates_df.to_sql('InterestRates', conn, index=False, if_exists='replace')
Yield_Curve_df.to_sql('YieldCurve', conn, index=False, if_exists='replace')
Housing_Starts_df.to_sql('HousingStarts', conn, index=False, if_exists='replace')
Jobless_Claims_df.to_sql('JoblessClaims', conn, index=False, if_exists='replace')
Jobs_Added_df.to_sql('JobsAdded', conn, index=False, if_exists='replace')

# ---------------------------
# 📊 SQL Query to Join Tables
# ---------------------------
sql_query = """
SELECT *
FROM InterestRates
LEFT JOIN BusinessConfidence ON InterestRates.observation_date = BusinessConfidence.observation_date
LEFT JOIN ConsumerConfidence ON InterestRates.observation_date = ConsumerConfidence.observation_date
LEFT JOIN OilPrices ON InterestRates.observation_date = OilPrices.observation_date
LEFT JOIN YieldCurve ON InterestRates.observation_date = YieldCurve.observation_date
LEFT JOIN HousingStarts ON InterestRates.observation_date = HousingStarts.observation_date
LEFT JOIN JoblessClaims ON InterestRates.observation_date = JoblessClaims.observation_date
LEFT JOIN JobsAdded ON InterestRates.observation_date = JobsAdded.observation_date
WHERE InterestRates.observation_date >= '1978-01-01'
"""

# Execute the query and load results into a DataFrame
Leading_Indicators_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Leading/Leading_Indicators_df.csv"
Leading_Indicators_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Provide the final DataFrame
Leading_Indicators_df


,observation_date,DFF,observation_date,BSCICP02USM460S,observation_date,UMCSENT,observation_date,WTISPLC,observation_date,T10Y2Y,observation_date,HOUST,observation_date,ICSA,observation_date,PAYEMS
0,1978-01-01,6.76,1978-01-01,14.8,1978-01-01,83.7,1978-01-01,14.85,1978-01-01,0.47,1978-01-01,1718,1978-01-01,361083,1978-01-01,84594
1,1978-04-01,7.28,1978-04-01,15.4,1978-04-01,81.6,1978-04-01,14.85,1978-04-01,0.41,1978-04-01,2197,1978-04-01,325769,1978-04-01,86162
2,1978-07-01,8.09,1978-07-01,24.4,1978-07-01,82.4,1978-07-01,14.85,1978-07-01,0.15,1978-07-01,2092,1978-07-01,339929,1978-07-01,87204
3,1978-10-01,9.58,1978-10-01,20.2,1978-10-01,79.3,1978-10-01,14.85,1978-10-01,-0.21,1978-10-01,1981,1978-10-01,339462,1978-10-01,87956
4,1979-01-01,10.07,1979-01-01,17.0,1979-01-01,72.1,1979-01-01,14.85,1979-01-01,-0.76,1979-01-01,1630,1979-01-01,358000,1979-01-01,88808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2024-01-01,5.33,2024-01-01,-1.8,2024-01-01,79.0,2024-01-01,74.15,2024-01-01,-0.26,2024-01-01,1376,2024-01-01,212154,2024-01-01,157049
185,2024-04-01,5.33,2024-04-01,-2.4,2024-04-01,77.2,2024-04-01,85.35,2024-04-01,-0.33,2024-04-01,1377,2024-04-01,223615,2024-04-01,157635
186,2024-07-01,5.26,2024-07-01,-6.0,2024-07-01,66.4,2024-07-01,81.80,2024-07-01,-0.25,2024-07-01,1262,2024-07-01,230769,2024-07-01,158003
187,2024-10-01,4.65,2024-10-01,-6.2,2024-10-01,70.5,2024-10-01,71.99,2024-10-01,0.12,2024-10-01,1344,2024-10-01,225462,2024-10-01,158358


## Lagging Indicators
| File Name | Type | Use |
|-----------|------|-----|
| `PPI_Inflation_Rate.csv` | Lagging | Feature |
| `CPI_Inflation_Rate.csv` | Lagging | Feature |
| `Unemployment_Rate.csv` | Lagging | Feature |
| `Labor_Force_Participation.csv` | Lagging | Feature |
| `Real_Gross_Domestic_Product.csv` | Lagging | **Label source** |
| `Deficit_Percent_GDP.csv` | Lagging | Feature (used in derived fiscal stress index) |


In [103]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
CPI_Inflation_df = pd.read_csv('Lagging/CPI_Inflation_Rate.csv')
Labor_Force_df = pd.read_csv('Lagging/Labor_Force_Participation.csv')
PPI_Inflation_df = pd.read_csv('Lagging/PPI_Inflation_Rate.csv')
GDP_df = pd.read_csv('Lagging/Real_Gross_Domestic_Product.csv')
Unemployment_df = pd.read_csv('Lagging/Unemployment_Rate.csv')


conn = sqlite3.connect(':memory:')

CPI_Inflation_df.to_sql('CPI', conn, index=False, if_exists='replace')
Labor_Force_df.to_sql('Labor_Force', conn, index=False, if_exists='replace')
PPI_Inflation_df.to_sql('PPI', conn, index=False, if_exists='replace')
GDP_df.to_sql('GDP', conn, index=False, if_exists='replace')
Unemployment_df.to_sql('Unemployment', conn, index=False, if_exists='replace')


# ---------------------------
# 📊 SQL Query to Join Tables
# ---------------------------
sql_query = """
SELECT *
FROM GDP
LEFT JOIN CPI ON GDP.observation_date = CPI.observation_date
LEFT JOIN Labor_Force ON GDP.observation_date = Labor_Force.observation_date
LEFT JOIN PPI ON GDP.observation_date = PPI.observation_date
LEFT JOIN Unemployment ON GDP.observation_date = Unemployment.observation_date

WHERE GDP.observation_date >= '1978-01-01'
"""

# Execute the query and load results into a DataFrame
Lagging_Indicators_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Lagging/Lagging_Indicators_df.csv"
Lagging_Indicators_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Provide the final DataFrame
Lagging_Indicators_df

,observation_date,GDPC1,observation_date,CPIAUCSL,observation_date,CIVPART,observation_date,PPIACO,observation_date,UNRATE
0,1978-01-01,6796.260,1978-01-01,62.700,1978-01-01,62.8,1978-01-01,66.800,1978-01-01,6.4
1,1978-04-01,7058.920,1978-04-01,63.900,1978-04-01,63.0,1978-04-01,69.000,1978-04-01,6.1
2,1978-07-01,7129.915,1978-07-01,65.500,1978-07-01,63.2,1978-07-01,70.400,1978-07-01,6.2
3,1978-10-01,7225.750,1978-10-01,67.100,1978-10-01,63.3,1978-10-01,71.800,1978-10-01,5.8
4,1979-01-01,7238.727,1979-01-01,68.500,1979-01-01,63.6,1979-01-01,73.800,1979-01-01,5.9
...,...,...,...,...,...,...,...,...,...,...
183,2023-10-01,22960.600,2023-10-01,307.653,2023-10-01,62.7,2023-10-01,255.192,2023-10-01,3.9
184,2024-01-01,23053.545,2024-01-01,309.794,2024-01-01,62.5,2024-01-01,251.306,2024-01-01,3.7
185,2024-04-01,23223.906,2024-04-01,313.016,2024-04-01,62.7,2024-04-01,256.978,2024-04-01,3.9
186,2024-07-01,23400.294,2024-07-01,313.566,2024-07-01,62.7,2024-07-01,257.321,2024-07-01,4.2


## Coincident Indicators

| File Name | Type | Use |
|-----------|------|-----|
| `Capacity_Utilization_Index.csv` | Coincident / Leading | Feature + Label Logic |
| `Industrial_Production_Index.csv` | Coincident | Feature |


In [104]:
import pandas as pd
import sqlite3

# Load CSV files into DataFrames
Capacity_df = pd.read_csv('Coincident/Capacity_Utilization_Index.csv')
Industrial_Production_df = pd.read_csv('Coincident/Industrial_Production_Index.csv')

conn = sqlite3.connect(':memory:')

Capacity_df.to_sql('Capacity', conn, index=False, if_exists='replace')
Industrial_Production_df.to_sql('IndProd', conn, index=False, if_exists='replace')



sql_query = """
SELECT *
FROM Capacity
LEFT JOIN IndProd ON Capacity.observation_date = IndProd.observation_date

WHERE Capacity.observation_date >= '1978-01-01'
"""

# Execute the query and load results into a DataFrame
Coincident_Indicators_df = pd.read_sql_query(sql_query, conn)

# Save the merged DataFrame to a new CSV file
csv_filename = "Coincident/Coincident_Indicators_df.csv"
Coincident_Indicators_df.to_csv(csv_filename, index=False)

# Close the connection
conn.close()

# Provide the final DataFrame
Coincident_Indicators_df

,observation_date,TCU,observation_date,INDPRO
0,1978-01-01,82.8849,1978-01-01,47.7512
1,1978-04-01,85.5183,1978-04-01,49.9074
2,1978-07-01,85.5628,1978-07-01,50.4371
3,1978-10-01,86.3582,1978-10-01,51.1523
4,1979-01-01,86.0650,1979-01-01,51.5250
...,...,...,...,...
183,2023-10-01,78.2766,2023-10-01,102.5781
184,2024-01-01,77.7036,2024-01-01,101.4830
185,2024-04-01,77.9731,2024-04-01,102.3568
186,2024-07-01,77.6310,2024-07-01,102.5192


## Renaming Columns

In [105]:

Final_Lagging_Indicators_df = Lagging_Indicators_df.loc[:, ~Lagging_Indicators_df.columns.duplicated()]
Final_Lagging_Indicators_df.columns = [
    'Date',
    'Real_GDP',
    'CPI_Inflation',
    'Labor_Force_Participation',
    'PPI_Inflation',
    'Unemployment_Rate'
]

Final_Lagging_Indicators_df.head()

,Date,Real_GDP,CPI_Inflation,Labor_Force_Participation,PPI_Inflation,Unemployment_Rate
0,1978-01-01,6796.260,62.7,62.8,66.8,6.4
1,1978-04-01,7058.920,63.9,63.0,69.0,6.1
2,1978-07-01,7129.915,65.5,63.2,70.4,6.2
3,1978-10-01,7225.750,67.1,63.3,71.8,5.8
4,1979-01-01,7238.727,68.5,63.6,73.8,5.9


In [107]:
Final_Leading_Indicators_df = Leading_Indicators_df.loc[:, ~Leading_Indicators_df.columns.duplicated()]
Final_Leading_Indicators_df.columns = [
    'Date',
    'Fed_Funds_Rate',
    'Business_Sentiment',
    'Consumer_Sentiment',
    'Oil_Price_WTI',
    'Yield_Curve_10Y_2Y',
    'Housing_Starts',
    'Jobless_Claims',
    'Nonfarm_Payrolls',
]
Final_Leading_Indicators_df.head()

,Date,Fed_Funds_Rate,Business_Sentiment,Consumer_Sentiment,Oil_Price_WTI,Yield_Curve_10Y_2Y,Housing_Starts,Jobless_Claims,Nonfarm_Payrolls
0,1978-01-01,6.76,14.8,83.7,14.85,0.47,1718,361083,84594
1,1978-04-01,7.28,15.4,81.6,14.85,0.41,2197,325769,86162
2,1978-07-01,8.09,24.4,82.4,14.85,0.15,2092,339929,87204
3,1978-10-01,9.58,20.2,79.3,14.85,-0.21,1981,339462,87956
4,1979-01-01,10.07,17.0,72.1,14.85,-0.76,1630,358000,88808


In [108]:
Final_Coincident_Indicators_df = Coincident_Indicators_df.loc[:, ~Coincident_Indicators_df.columns.duplicated()]
Final_Coincident_Indicators_df.columns = [
    'Date',
    'Capacity_Utilization',
    'Industrial_Production'
]
Final_Coincident_Indicators_df.head()

,Date,Capacity_Utilization,Industrial_Production
0,1978-01-01,82.8849,47.7512
1,1978-04-01,85.5183,49.9074
2,1978-07-01,85.5628,50.4371
3,1978-10-01,86.3582,51.1523
4,1979-01-01,86.0650,51.5250


## Analyizing Lagging, Leading, & Coincident 4 quarters before 1980, 1981, 1990, 2001, 2008, 2020

### 1980 Recession (Start= January 1980)

In [120]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1979-01-01'
AND LD.date <= '1981-01-01'
"""


# Run the query
Pre_1980_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1980_Recession.csv"
Pre_1980_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1980_Recession


,Date,Fed_Funds_Rate,Business_Sentiment,Consumer_Sentiment,Oil_Price_WTI,Yield_Curve_10Y_2Y,Housing_Starts,Jobless_Claims,Nonfarm_Payrolls,Date,Real_GDP,CPI_Inflation,Labor_Force_Participation,PPI_Inflation,Unemployment_Rate,Date,Capacity_Utilization,Industrial_Production
0,1979-01-01,10.07,17.0,72.1,14.85,-0.76,1630,358000,88808,1979-01-01,7238.727,68.5,63.6,73.8,5.9,1979-01-01,86.0650,51.5250
1,1979-04-01,10.18,12.4,66.0,15.85,-0.60,1748,373385,89417,1979-04-01,7246.454,70.6,63.5,76.9,5.8,1979-04-01,85.1806,51.3251
2,1979-07-01,10.94,2.6,60.4,21.75,-0.19,1760,386231,90217,1979-07-01,7300.281,73.0,63.6,79.2,5.7,1979-07-01,84.2881,51.7029
3,1979-10-01,13.58,-2.0,62.1,29.00,-1.19,1681,415615,90481,1979-10-01,7318.535,75.2,63.7,82.1,6.0,1979-10-01,84.0443,51.7175
4,1980-01-01,15.07,-7.6,67.0,32.50,-0.70,1341,422923,90800,1980-01-01,7341.557,78.0,64.0,85.2,6.3,1980-01-01,83.8941,51.9440
5,1980-04-01,12.67,-25.2,52.7,39.50,-1.03,1051,586615,90849,1980-04-01,7190.289,80.9,63.8,87.8,6.9,1980-04-01,79.8992,50.7197
6,1980-07-01,9.82,-30.0,62.3,39.50,1.22,1269,523769,89840,1980-07-01,7181.743,82.6,63.8,90.3,7.8,1980-07-01,78.1646,48.4907
7,1980-10-01,15.85,11.0,75.0,36.00,-0.34,1523,419615,90490,1980-10-01,7315.677,84.7,63.7,92.8,7.5,1980-10-01,80.7126,50.0861
8,1981-01-01,16.60,-1.6,71.4,38.00,-0.69,1547,416154,91033,1981-01-01,7459.022,87.2,63.9,95.2,7.5,1981-01-01,80.3913,50.9328


### 1981 Recession (Start= July 1981)

In [121]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1980-07-01'
AND LD.date <= '1982-07-01'
"""


# Run the query
Pre_1981_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1981_Recession.csv"
Pre_1981_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1981_Recession


,Date,Fed_Funds_Rate,Business_Sentiment,Consumer_Sentiment,Oil_Price_WTI,Yield_Curve_10Y_2Y,Housing_Starts,Jobless_Claims,Nonfarm_Payrolls,Date,Real_GDP,CPI_Inflation,Labor_Force_Participation,PPI_Inflation,Unemployment_Rate,Date,Capacity_Utilization,Industrial_Production
0,1980-07-01,9.82,-30.0,62.3,39.50,1.22,1269,523769,89840,1980-07-01,7181.743,82.6,63.8,90.3,7.8,1980-07-01,78.1646,48.4907
1,1980-10-01,15.85,11.0,75.0,36.00,-0.34,1523,419615,90490,1980-10-01,7315.677,84.7,63.7,92.8,7.5,1980-10-01,80.7126,50.0861
2,1981-01-01,16.60,-1.6,71.4,38.00,-0.69,1547,416154,91033,1981-01-01,7459.022,87.2,63.9,95.2,7.5,1981-01-01,80.3913,50.9328
3,1981-04-01,17.79,3.2,72.4,38.00,-0.47,1360,419615,91283,1981-04-01,7403.745,89.1,64.2,98.0,7.2,1981-04-01,80.0991,50.7661
4,1981-07-01,17.59,-6.6,74.1,36.00,-1.07,1041,451308,91601,1981-07-01,7492.405,91.5,63.8,99.0,7.2,1981-07-01,80.2781,51.6555
5,1981-10-01,13.59,-20.0,70.3,35.00,-0.39,873,516538,91380,1981-10-01,7410.768,93.4,63.8,98.9,7.9,1981-10-01,77.9184,50.9463
6,1982-01-01,14.21,-23.6,71.0,33.85,0.03,843,541308,90565,1982-01-01,7295.631,94.4,63.7,99.7,8.6,1982-01-01,75.8154,48.7877
7,1982-04-01,14.51,-24.4,65.5,33.45,-0.33,917,589615,90150,1982-04-01,7328.912,95.0,63.9,99.6,9.3,1982-04-01,74.4429,48.9913
8,1982-07-01,11.01,-23.2,65.4,34.16,0.15,1166,618538,89521,1982-07-01,7300.896,97.5,64.0,100.4,9.8,1982-07-01,73.0408,48.3775


### 1990 Recession (Start= July 1990)

In [122]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '1989-07-01'
AND LD.date <= '1991-07-01'
"""


# Run the query
Pre_1990_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1990_Recession.csv"
Pre_1990_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_1990_Recession


,Date,Fed_Funds_Rate,Business_Sentiment,Consumer_Sentiment,Oil_Price_WTI,Yield_Curve_10Y_2Y,Housing_Starts,Jobless_Claims,Nonfarm_Payrolls,Date,Real_GDP,CPI_Inflation,Labor_Force_Participation,PPI_Inflation,Unemployment_Rate,Date,Capacity_Utilization,Industrial_Production
0,1989-07-01,9.08,-8.2,92.0,19.644,0.20,1427,337571,108069,1989-07-01,9919.228,124.5,66.5,112.8,5.2,1989-07-01,83.0423,61.0896
1,1989-10-01,8.61,-6.4,93.9,20.091,0.03,1410,352000,108476,1989-10-01,9938.767,125.4,66.5,112.8,5.3,1989-10-01,82.7612,61.4014
2,1990-01-01,8.25,-5.6,93.0,22.641,0.12,1551,354077,109196,1990-01-01,10047.386,127.5,66.8,114.9,5.4,1990-01-01,82.7599,61.6352
3,1990-04-01,8.24,0.0,93.9,18.582,0.06,1248,360000,109674,1990-04-01,10083.855,128.9,66.6,114.1,5.4,1990-04-01,82.7825,62.3511
4,1990-07-01,8.16,-6.8,88.2,18.638,0.31,1171,382692,109815,1990-07-01,10090.569,130.5,66.5,114.5,5.5,1990-07-01,82.6474,62.6435
5,1990-10-01,7.74,-13.6,63.9,35.922,0.84,1014,443308,109377,1990-10-01,9998.704,133.4,66.4,120.8,5.9,1990-10-01,81.0212,62.4973
6,1991-01-01,6.43,-21.6,66.8,24.959,0.97,798,476000,109058,1991-01-01,9951.916,134.7,66.2,119.0,6.4,1991-01-01,79.2415,61.0842
7,1991-04-01,5.86,-14.4,81.8,20.823,1.09,1001,446154,108350,1991-04-01,10029.510,135.1,66.4,116.0,6.7,1991-04-01,79.4862,60.4524
8,1991-07-01,5.65,1.2,82.9,21.420,1.36,1063,425308,108290,1991-07-01,10080.195,136.2,66.1,116.1,6.8,1991-07-01,80.3995,61.7201


### 2001 Recession (Start= March 2001)

In [123]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2000-04-01'
AND LD.date <= '2002-04-01'
"""


# Run the query
Pre_2001_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_1990_Recession.csv"
Pre_2001_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2001_Recession

,Date,Fed_Funds_Rate,Business_Sentiment,Consumer_Sentiment,Oil_Price_WTI,Yield_Curve_10Y_2Y,Housing_Starts,Jobless_Claims,Nonfarm_Payrolls,Date,Real_GDP,CPI_Inflation,Labor_Force_Participation,PPI_Inflation,Unemployment_Rate,Date,Capacity_Utilization,Industrial_Production
0,2000-04-01,6.27,9.4,109.2,25.74,-0.41,1626,280846,131883,2000-04-01,14130.908,170.9,67.3,130.7,3.8,2000-04-01,82.2761,92.6659
1,2000-07-01,6.52,5.0,108.3,29.77,-0.29,1463,301929,132228,2000-07-01,14145.312,172.7,66.9,133.7,4.0,2000-07-01,81.4061,92.8373
2,2000-10-01,6.47,-2.6,105.8,33.08,-0.17,1549,328538,132352,2000-10-01,14229.765,173.9,66.8,135.4,3.9,2000-10-01,80.3854,92.6400
3,2001-01-01,5.60,-17.2,94.7,29.58,0.40,1600,367769,132703,2001-01-01,14183.120,175.6,67.2,140.0,4.2,2001-01-01,78.6272,91.8908
4,2001-04-01,4.33,-13.8,88.4,27.41,0.91,1649,396231,132457,2001-04-01,14271.694,176.4,66.9,136.4,4.4,2001-04-01,76.9291,90.7384
5,2001-07-01,3.50,-10.6,92.4,26.45,1.20,1670,412231,132171,2001-07-01,14214.516,177.4,66.8,133.4,4.6,2001-07-01,75.2728,89.2352
6,2001-10-01,2.13,-19.0,82.7,22.21,1.84,1540,445923,131452,2001-10-01,14253.574,177.6,66.7,130.3,5.3,2001-10-01,73.9587,88.4051
7,2002-01-01,1.73,-1.8,93.0,19.67,2.01,1698,408231,130859,2002-01-01,14372.785,177.7,66.5,128.5,5.7,2002-01-01,74.1248,88.4634
8,2002-04-01,1.75,8.8,93.0,26.27,1.79,1592,406846,130615,2002-04-01,14460.848,179.3,66.7,130.8,5.9,2002-04-01,75.0518,89.5507


### 2008 Recession (Start= December 2007)

In [124]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2007-01-01'
AND LD.date <= '2009-01-01'
"""


# Run the query
Pre_2008_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_2008_Recession.csv"
Pre_2008_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2008_Recession

,Date,Fed_Funds_Rate,Business_Sentiment,Consumer_Sentiment,Oil_Price_WTI,Yield_Curve_10Y_2Y,Housing_Starts,Jobless_Claims,Nonfarm_Payrolls,Date,Real_GDP,CPI_Inflation,Labor_Force_Participation,PPI_Inflation,Unemployment_Rate,Date,Capacity_Utilization,Industrial_Production
0,2007-01-01,5.25,-1.4,96.9,54.57,-0.12,1409,315923,137472,2007-01-01,16611.690,203.437,66.4,164.0,4.6,2007-01-01,80.5114,99.7546
1,2007-04-01,5.25,5.6,87.1,63.97,0.03,1490,313385,137845,2007-04-01,16713.314,205.904,65.9,171.4,4.5,2007-04-01,80.9348,101.5723
2,2007-07-01,5.07,4.6,90.4,74.18,0.19,1354,315385,138044,2007-07-01,16809.587,207.603,66.0,175.1,4.7,2007-07-01,80.7002,101.4953
3,2007-10-01,4.50,0.8,80.9,86.20,0.56,1264,338308,138177,2007-10-01,16915.191,209.190,65.8,174.7,4.7,2007-10-01,80.9210,101.6446
4,2008-01-01,3.18,1.4,78.4,92.95,1.27,1084,350692,138391,2008-01-01,16843.003,212.174,66.2,181.0,5.0,2008-01-01,80.8616,102.1493
5,2008-04-01,2.08,-2.8,62.6,112.57,1.63,1013,369615,138038,2008-04-01,16943.291,213.942,65.9,190.9,5.0,2008-04-01,79.8024,100.7554
6,2008-07-01,1.94,0.0,61.2,133.44,1.43,923,431769,137498,2008-07-01,16854.295,219.016,66.1,205.5,5.8,2008-07-01,77.1346,99.4194
7,2008-10-01,0.51,-22.2,57.6,76.65,2.20,777,519692,136293,2008-10-01,16485.350,216.995,66.0,186.4,6.5,2008-10-01,73.5965,94.4985
8,2009-01-01,0.18,-28.8,61.2,41.74,1.71,490,621000,134078,2009-01-01,16298.262,211.933,65.7,171.2,7.8,2009-01-01,69.1611,88.3712


In [119]:
import pandas as pd
import sqlite3

# Create in-memory SQLite connection
conn = sqlite3.connect(':memory:')

# Assume these DataFrames are already loaded:
# Final_Coincident_Indicators_df, Final_Leading_Indicators_df, Final_Lagging_Indicators_df

# Save them to the SQLite database
Final_Coincident_Indicators_df.to_sql('Coincident', conn, index=False, if_exists='replace')
Final_Leading_Indicators_df.to_sql('Leading', conn, index=False, if_exists='replace')
Final_Lagging_Indicators_df.to_sql('Lagging', conn, index=False, if_exists='replace')

# Proper SQL query
sql_query = """
SELECT *
FROM Leading LD
JOIN Lagging LG ON LD.date = LG.date
JOIN Coincident C ON LD.date = C.date
WHERE LD.date >= '2019-04-01'
AND LD.date <= '2020-04-01'
"""


# Run the query
Pre_2020_Recession = pd.read_sql_query(sql_query, conn)

# Save to CSV
csv_filename = "Pre_2020_Recession.csv"
Pre_2020_Recession.to_csv(csv_filename, index=False)

# Close DB connection
conn.close()

# Show final DataFrame
Pre_2020_Recession

,Date,Fed_Funds_Rate,Business_Sentiment,Consumer_Sentiment,Oil_Price_WTI,Yield_Curve_10Y_2Y,Housing_Starts,Jobless_Claims,Nonfarm_Payrolls,Date,Real_GDP,CPI_Inflation,Labor_Force_Participation,PPI_Inflation,Unemployment_Rate,Date,Capacity_Utilization,Industrial_Production
0,2019-04-01,2.40,5.6,97.2,63.86,0.19,1272,218385,150593,2019-04-01,20602.275,255.233,62.8,202.1,3.7,2019-04-01,78.7309,102.2748
1,2019-07-01,2.19,2.4,98.4,57.35,0.22,1240,209769,150936,2019-07-01,20843.322,255.802,63.1,200.7,3.7,2019-07-01,78.5013,102.0057
2,2019-10-01,1.65,-3.4,95.5,53.96,0.16,1325,224231,151460,2019-10-01,20985.448,257.155,63.3,198.6,3.6,2019-10-01,77.9124,101.5878
3,2020-01-01,1.25,1.8,99.8,57.52,0.24,1578,865692,152031,2020-01-01,20693.238,259.127,63.3,199.3,3.6,2020-01-01,76.5390,101.3372
4,2020-04-01,0.06,-17.0,71.8,16.55,0.43,931,2729308,130424,2020-04-01,19056.617,256.032,60.1,185.5,14.8,2020-04-01,66.8603,84.6812
